# Non-Stationary Section Code
This notebook is meant for holding only non-stationary related exploration.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Dict

import seaborn as sns

In [2]:
import numpy as np
from tigramite import data_processing as pp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr

In [3]:
import concurrent.futures


## Pre-processing

In [4]:
DATA_DIR = 'data'
INPUT_DIR = 'input'
OUTPUT_DIR = 'output'
INPUT_PATH = f'{DATA_DIR}/{INPUT_DIR}'
OUTPUT_PATH = f'{DATA_DIR}/{OUTPUT_DIR}'

In [5]:
df = pd.read_csv(f'{INPUT_PATH}/sectoral_fundamentals.csv')
sector_column = 'gicdesc'
date_column = 'public_date'
df[date_column] = pd.to_datetime(df[date_column])
df = df.set_index([sector_column, date_column])

In [6]:
percent_cols = [col for col in df.columns if df[col].dtype == "object" and df[col].str.contains('%').any()]
for col in percent_cols:
    df[col] = df[col].str.rstrip('%').astype('float') / 100

In [7]:
target_columns = ['simulated_portfolio_value']
returns_col = ['indret_ew']
initial_portafolio_value = 10000
lookahead_period = 1

#### Create Simulated portfolio

In [8]:
def get_simulated_portfolio(df : pd.DataFrame, target_columns : list, returns_col : list, initial_portafolio_value=1000, lookahead_period=0) -> pd.DataFrame:
    df = df.sort_index(ascending=True)

    # Convert MoM percentage change to multiplicative factors
    df['multiplicative_factor'] = 1 + df[returns_col]
    # Calculate the cumulative product of multiplicative factors
    df['cumulative_factor'] = df['multiplicative_factor'].cumprod()

    # Calculate the market capitalization using the cumulative factor
    df[target_columns[0]] = initial_portafolio_value * df['cumulative_factor']
    df[target_columns] = df[target_columns].shift(-lookahead_period)

    # indret_vw hard coded, but does not matter.
    df = df.drop(columns=['multiplicative_factor', 'cumulative_factor', 'indret_vw'] + returns_col).dropna()

    return df

In [9]:
source_df = df.copy()

In [10]:
sector_dict = {sector:{'source': get_simulated_portfolio(data, target_columns,returns_col, initial_portafolio_value,lookahead_period)} for sector, data in df.groupby(sector_column)}

In [11]:
og_results = {sector:data[returns_col] for sector, data in source_df.groupby(sector_column)}

In [12]:
required_structure = {
    'sector': {
        'source':pd.DataFrame,
        'data_splits': {
            'total_size' : int,
            'train_size': int,
            'test_size': int,
            'holdout_size': int,
        },
        'feature_selection' : {
            'pcmci+': {
                'test': pd.DataFrame,
                'holdout': pd.DataFrame,
                'train': pd.DataFrame,
                'features_lags': {},
                'raw_results':{}
            }
        }
    }
}

In [13]:
def pcmci_plus_results(df_tigramite : pp.DataFrame, verbosity : int = 1, max_lag : int = 12, alpha_level = 0.05):
    # Initialize PCMCI object
    pcmci = PCMCI(dataframe=df_tigramite, cond_ind_test=ParCorr(), verbosity=verbosity)

    # Compute the p-values
    return pcmci.run_pcmciplus(tau_max=max_lag, pc_alpha=alpha_level)

In [14]:
def process_pcmci_results(results, var_names, target_var, alpha_level=0.05):

    # Get the results for the target variable
    p_matrix = results['p_matrix']
    val_matrix = results['val_matrix']
    # sig_matrix = np.multiply(p_matrix < 0.05, val_matrix)

    # Get the results for the target variable
    target_results = p_matrix[var_names.index(target_var), :]
    target_results = pd.DataFrame(target_results, index=var_names)

    reshaped_df = target_results[target_results.abs() < alpha_level].reset_index().melt(id_vars='index' , var_name='lag', value_name='p-value').dropna().rename(columns={'index': 'column'})
    sorted_list = reshaped_df.sort_values(by='p-value', key=abs)
    sorted_list['value_str'] = sorted_list['p-value'].apply(lambda x: f'{x:.6f}') 
    # print(sorted_list.head(25).to_latex())

    grouped_values = sorted_list.groupby('column')['lag'].apply(list).to_dict()
    return grouped_values

In [15]:
def get_pcmci_plus_features(df : pd.DataFrame, features_to_check, target_var : str, max_lag : int = 12, alpha_level=0.05, verbosity=1) -> dict:
    # Prepare the dataframe
    var_names = features_to_check + [target_var]
    df_tigramite = pp.DataFrame(df[var_names].values, var_names=var_names)

    raw_results = pcmci_plus_results(df_tigramite, verbosity=verbosity, max_lag=max_lag, alpha_level=alpha_level)
    lag_dict = process_pcmci_results(raw_results, var_names, target_var, alpha_level=alpha_level)

    return {
        'raw_results': raw_results,
        'features_lags': lag_dict
    }

In [16]:
def get_correlation_features(df : pd.DataFrame, features_to_check, target_var : str, top_n : int = 10) -> dict:
    corr = df[features_to_check + [target_var]].corr().abs().sort_values(target_var, ascending=False)
    raw_results = corr.index[1:top_n+1].tolist()
    features_lags = {feature:[0] for feature in raw_results}
    return {
        'raw_results': raw_results,
        'features_lags': features_lags
    }

In [17]:
def get_lagged_correlation(df : pd.DataFrame, feature : str, target_var : str, max_lag = 12) -> dict:
    df = df.copy()
    if feature == target_var:
        init_num = 1
        feature = feature + '_'
        df[feature] = df[target_var]
    else:
        init_num = 0

    feature_results = {feature:{}}
    for lag in range(init_num, max_lag +1):
        curr_df = df[[feature, target_var]].copy()
        curr_df[feature] = curr_df[feature].shift(lag)
        curr_df = curr_df.dropna()
        # Check if standard deviation is zero
        if curr_df[feature].std() == 0 or curr_df[target_var].std() == 0:
            feature_results[feature][lag] = np.nan
        else:
            feature_results[feature][lag] = np.abs(curr_df[feature].corr(curr_df[target_var]))
    return feature_results

def get_lagged_correlation_features(df : pd.DataFrame, features_to_check, target_var : str, top_n : int = 10, max_lag = 12, max_per_feature = 3) -> dict:
    df = df.copy()
    raw_results = []
    results = []
    for feature in features_to_check + [target_var]:
        feature_results = get_lagged_correlation(df, feature, target_var, max_lag)
        raw_results.append(feature_results)
        key = feature
        if feature == target_var:
            feature += '_'

        # Flatten the results and add them to the list
        for lag, corr in feature_results[feature].items():
            results.append((key, lag, corr))

    # Sort by absolute correlation in descending order and lag in ascending order
    results.sort(key=lambda x: (abs(x[2]), -x[1]), reverse=True)
    # Select top features, with a maximum of max_per_feature per feature
    final_results = {}
    feature_counts = {}
    total_features = 0
    for feature, lag, corr in results:
        if feature_counts.get(feature, 0) < max_per_feature:
            if feature not in final_results:
                final_results[feature] = []
                total_features += 1
            final_results[feature].append(lag)
            feature_counts[feature] = feature_counts.get(feature, 0) + 1
        if total_features == top_n:
            break
    return {
        'raw_results': raw_results,
        'features_lags': final_results
    }

In [18]:
# for sector in sector_dict:
#     df = sector_dict[sector]['source'].copy()
#     holdout_size = int(len(df) * 0.2)
#     sector_dict[sector]['holdout'] = df[-holdout_size:]
#     df = df[:-holdout_size]
#     test_size = int(len(df) * 0.2)
#     sector_dict[sector]['test'] = df[-test_size:]
#     sector_dict[sector]['train'] = df[:-test_size]

In [19]:
feature_selection_models = ['pcmci+', 'correlation_top_10', 'lagged_correlation_top_10']

In [20]:
max_lag = 12

exclude_features = ['rd_sale_Median', 'adv_sale_Median', 'staff_sale_Median', target_columns[0], 'indret_ew', 'indret_vw', 'multiplicative_factor']
features_to_check = [x for x in df.columns.to_list() if x not in exclude_features]

In [21]:
for sector in sector_dict:
    df = sector_dict[sector]['source'].copy()
    total_size = len(df)
    holdout_size = int(total_size * 0.2)
    test_size = int((total_size - holdout_size) * 0.2)
    sector_dict[sector]['data_splits'] = {
        'total_size': total_size,
        'holdout_size': holdout_size,
        'test_size': test_size,
        'train_size': total_size - test_size - holdout_size
    }
    sector_dict[sector]['feature_selection'] = {}
    feature_finder_df = df[:- test_size - holdout_size]
    for model in feature_selection_models:
        if model == 'pcmci+':
            sector_dict[sector]['feature_selection'][model] = get_pcmci_plus_features(feature_finder_df, features_to_check, target_columns[0], max_lag=max_lag, alpha_level=0.05, verbosity=1)
        elif model == 'correlation_top_10':
            sector_dict[sector]['feature_selection'][model] = get_correlation_features(feature_finder_df, features_to_check, target_columns[0], top_n=10)
        elif model == 'lagged_correlation_top_10':
            sector_dict[sector]['feature_selection'][model] = get_lagged_correlation_features(feature_finder_df, features_to_check, target_columns[0], top_n=10, max_lag=max_lag)


##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 12
pc_alpha = [0.05]
max_conds_dim = None
max_combinations = 1



## Resulting lagged parent (super)sets:

    Variable NFIRM has 5 link(s):
        (NFIRM -1): max_pval = 0.00000, |min_val| =  0.733
        (aftret_invcapx_Median -6): max_pval = 0.00002, |min_val| =  0.460
        (inv_turn_Median -8): max_pval = 0.03005, |min_val| =  0.247
        (pe_inc_Median -2): max_pval = 0.04059, |min_val| =  0.232
        (pe_exi_Median -2): max_pval = 0.04902, |min_val| =  0.224

    Variable PEG_1yrforward_Median has 3 link(s):
        (PEG_1yrforward_Median -1): max_pval = 0.00000, |min_val| =  0.636
        (cfm_Median -3): max_pval = 0.03357, |min_val| =  0.241
        (ptb_Median -4): max_pval = 0.03258, |min_val| =  0.241

    Variable CAPEI_Median has 4 link(s):
        (CAPEI_Median -1): max_pval = 0.00000, |min_val| =  0.587
        (cash_lt_Median -9): max

/Users/marcello/Documents/GitHub/uva-msc-ds-thesis/.venv/lib/python3.10/site-packages/tigramite/data_processing.py:308: UserWarning: In analysis mode 'single', 'data'.shape = (49, 70); is it of shape (observations, variables)?
  warnings.warn("In analysis mode 'single', 'data'.shape = ({}, {});"\



##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 12
pc_alpha = [0.05]
max_conds_dim = None
max_combinations = 1



## Resulting lagged parent (super)sets:

    Variable NFIRM has 4 link(s):
        (profit_lct_Median -12): max_pval = 0.00609, |min_val| =  0.564
        (ocf_lct_Median -10): max_pval = 0.02169, |min_val| =  0.486
        (at_turn_Median -4): max_pval = 0.02186, |min_val| =  0.486
        (pay_turn_Median -7): max_pval = 0.03505, |min_val| =  0.441

    Variable PEG_1yrforward_Median has 3 link(s):
        (profit_lct_Median -10): max_pval = 0.00168, |min_val| =  0.606
        (de_ratio_Median -1): max_pval = 0.01636, |min_val| =  0.475
        (at_turn_Median -5): max_pval = 0.03593, |min_val| =  0.439

    Variable CAPEI_Median has 3 link(s):
        (simulated_portfolio_value -1): max_pval = 0.02197, |min_val| =  0.465
        (PEG_trailing_Median -12): max_pval = 0.03586, |min_val| =  0.43

In [22]:
lis = []

for sector in sector_dict:
    for model in feature_selection_models:
        for feature in sector_dict[sector]['feature_selection'][model]['features_lags']:
            for lag in sector_dict[sector]['feature_selection'][model]['features_lags'][feature]:
                lis.append({'sector': sector, 'model': model, 'feature': feature, 'lag': lag})

sector_lags = pd.DataFrame(lis)

In [23]:
feature_counts = sector_lags.groupby(['model'])['feature'].value_counts()

mean_lags = sector_lags.groupby(['model','feature'])['lag'].mean()
mean_lags = np.ceil(mean_lags).astype(int)

features_metrics_df = pd.concat([mean_lags, feature_counts], join='inner', axis=1).sort_values('count', ascending=False)

In [24]:
def create_lagged_df(df : pd.DataFrame, features_lags : dict, target_columns : list) -> pd.DataFrame:
    df = df.copy()
    lagged_cols = []
    for feature in features_lags:
        for lag in features_lags[feature]:
            new_col = f'{feature}_lag_{lag}'
            df[new_col] = df[feature].shift(-lag)
            lagged_cols.append(new_col)
    df = df[lagged_cols + target_columns].dropna()
    return df

In [25]:
for sector in sector_dict:
    test_size = sector_dict[sector]['data_splits']['test_size']
    holdout_size = sector_dict[sector]['data_splits']['holdout_size']
    for model in feature_selection_models:
        sector_dict[sector]['feature_selection'][model]['lagged'] = create_lagged_df(sector_dict[sector]['source'], sector_dict[sector]['feature_selection'][model]['features_lags'], target_columns)
        sector_dict[sector]['feature_selection'][model]['train'] = sector_dict[sector]['feature_selection'][model]['lagged'][:- holdout_size] # -test_size 
        # commented because now we use holdout set for final evaluations
        # sector_dict[sector]['feature_selection'][model]['test'] = sector_dict[sector]['feature_selection'][model]['lagged'][-test_size - holdout_size:-holdout_size]
        sector_dict[sector]['feature_selection'][model]['holdout'] = sector_dict[sector]['feature_selection'][model]['lagged'][-holdout_size:]

## Fun Starts here

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [27]:
def evaluate_model(model, X_train, y_train, X_test, convert_to_variance=False, og_results=None, og_index=None):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if convert_to_variance:
        y_pred = pd.Series(y_pred.ravel()).pct_change().dropna()
        if og_index is not None:
            y_test = og_results.loc[og_index]
        else:
            y_test = og_results.loc[X_test[1:].index]
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    return mse, rmse, mae

In [28]:
def linear_regression_train(X_train, y_train):
    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    return model

In [29]:
def decision_tree_train(X_train, y_train):
    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)

    return model

In [30]:
def get_poly_x(X_train, X_test, degree=2):
    # Transform the features to polynomial
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    return X_train_poly, X_test_poly

In [31]:
predictive_models = ['linear_regression', 'decision_tree', 'polynomial_regression']

In [72]:
for sector in sector_dict:
    for feature_model in feature_selection_models:
        X_train = sector_dict[sector]['feature_selection'][feature_model]['train'].drop(columns=target_columns)
        y_train = sector_dict[sector]['feature_selection'][feature_model]['train'][target_columns]

        # X_test = sector_dict[sector]['feature_selection'][model]['test'].drop(columns=target_columns)
        # y_test = sector_dict[sector]['feature_selection'][model]['test'][target_columns]

        X_holdout = sector_dict[sector]['feature_selection'][feature_model]['holdout'].drop(columns=target_columns)
        y_holdout = sector_dict[sector]['feature_selection'][feature_model]['holdout'][target_columns]


        for predictive_model in predictive_models:
            if predictive_model == 'linear_regression':
                model = linear_regression_train(X_train, y_train)
            elif predictive_model == 'decision_tree':
                model = decision_tree_train(X_train, y_train)
            elif predictive_model == 'polynomial_regression':
                X_train_poly, X_holdout_poly = get_poly_x(X_train, X_holdout, degree=2)
                model = linear_regression_train(X_train_poly, y_train)
                mse, rmse, mae = evaluate_model(model, X_train_poly, y_train, X_holdout_poly, convert_to_variance=True, og_results=og_results[sector], og_index=X_holdout.index[1:])
                sector_dict[sector]['feature_selection'][feature_model]['polynomial_regression'] = {
                    'mse': mse,
                    'rmse': rmse,
                    'mae': mae,
                    'model': model
                }

            if predictive_model != 'polynomial_regression':
                mse, rmse, mae = evaluate_model(model, X_train, y_train, X_holdout, convert_to_variance=True, og_results=og_results[sector])
                sector_dict[sector]['feature_selection'][feature_model][predictive_model] = {
                    'mse': mse,
                    'rmse': rmse,
                    'mae': mae,
                    'model': model
                }

In [104]:
# Initialize a dictionary to store feature importances
feature_importances = {feature_model: {} for feature_model in feature_selection_models}

for sector in sector_dict:
    for feature_model in feature_selection_models:
        X_train = sector_dict[sector]['feature_selection'][feature_model]['train'].drop(columns=target_columns)
        for predictive_model in predictive_models:
            # Extract the trained model
            model = sector_dict[sector]['feature_selection'][feature_model][predictive_model]['model']
            
            # Extract feature importances or coefficients
            if predictive_model == 'decision_tree':
                importances = model.feature_importances_
            elif predictive_model in ['linear_regression', 'polynomial_regression']:
                importances = model.coef_
            else:
                continue

            for i in range(len(importances) -1):
                # Clean the feature name by removing the lag part
                feature = X_train.columns[i].rsplit('_lag_', 1)[0]
                importance = importances[i]
                if feature not in feature_importances[feature_model]:
                    feature_importances[feature_model][feature] = []
                feature_importances[feature_model][feature].append(importance)

# Aggregate feature importances for each feature_model
for feature_model in feature_importances:
    for feature in feature_importances[feature_model]:
        feature_importances[feature_model][feature] = sum(feature_importances[feature_model][feature]) / len(feature_importances[feature_model][feature])

In [106]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Create a list of colors
colors = list(mcolors.CSS4_COLORS.keys())

# Filter out light colors
colors = [color for color in colors if mcolors.to_rgb(color)[0] < 0.8 and mcolors.to_rgb(color)[1] < 0.8 and mcolors.to_rgb(color)[2] < 0.8]

# Map the feature_model to colors
feature_model_colors = {feature_model: colors[i % len(colors)] for i, feature_model in enumerate(feature_importances.keys())}

# Plot the feature importances for each feature_model
for i, feature_model in enumerate(feature_importances):
    # Sort the features by importance
    sorted_features = sorted(feature_importances[feature_model].items(), key=lambda x: x[1], reverse=True)

    # Limit the number of features to the top 10
    sorted_features = sorted_features[:10]

    # Separate the feature names and importances
    features, importances = zip(*sorted_features)

    # Create a new figure for each plot
    fig, ax = plt.subplots(figsize=(10, 10))

    # Create the bar chart with 80% opacity
    ax.barh(features, importances, color=feature_model_colors[feature_model], alpha=0.8)

    # Invert the y-axis to have the most important feature at the top
    ax.invert_yaxis()

    # Rotate the y-axis labels by 45 degrees
    plt.yticks(rotation=45)

    # Add labels and title
    ax.set_xlabel('Importance')
    ax.set_title(f'Feature Importances for {feature_model}')

    # Save the plot as an image file
    fig.savefig(f'misc/images/feature_importances_{feature_model}.png')

    # Close the figure to free up memory
    plt.close(fig)

In [33]:
lis = []

for sector in sector_dict:
    for model in feature_selection_models:
        for predictive_model in predictive_models:
            lis.append({'sector': sector, 
                        'model': model, 
                        'predictive_model': predictive_model, 
                        'mse': sector_dict[sector]['feature_selection'][model][predictive_model]['mse'], 
                        'rmse': sector_dict[sector]['feature_selection'][model][predictive_model]['rmse'], 
                        'mae': sector_dict[sector]['feature_selection'][model][predictive_model]['mae']
                        })

sector_model_metrics = pd.DataFrame(lis)

In [37]:
sector_model_metrics.sort_values(['sector', 'predictive_model', 'model'], ascending=[True, True, True])

,sector,model,predictive_model,mse,rmse,mae
4,Communication Services,correlation_top_10,decision_tree,0.013548,0.116398,0.078886
7,Communication Services,lagged_correlation_top_10,decision_tree,0.022031,0.148429,0.096727
1,Communication Services,pcmi+,decision_tree,0.008698,0.093265,0.072998
3,Communication Services,correlation_top_10,linear_regression,0.008256,0.090863,0.068412
6,Communication Services,lagged_correlation_top_10,linear_regression,0.011925,0.109203,0.085504
...,...,...,...,...,...,...
96,Utilities,lagged_correlation_top_10,linear_regression,0.003329,0.057700,0.051320
90,Utilities,pcmi+,linear_regression,0.004041,0.063569,0.050390
95,Utilities,correlation_top_10,polynomial_regression,0.023670,0.153852,0.107210
98,Utilities,lagged_correlation_top_10,polynomial_regression,0.208480,0.456596,0.340936


In [51]:
tt_df = sector_model_metrics.sort_values(['sector', 'predictive_model', 'model'], ascending=[True, True, True])
pivot_df = tt_df.pivot_table(index=['sector','model'], columns=['predictive_model'])

In [52]:
pivot_df.columns = pivot_df.columns.swaplevel(0, 1)
pivot_df.sort_index(axis=1, level=0, inplace=True)
pivot_df

predictive_model                                 decision_tree            \
                                                           mae       mse   
sector                 model                                               
Communication Services correlation_top_10             0.078886  0.013548   
                       lagged_correlation_top_10      0.096727  0.022031   
                       pcmi+                          0.072998  0.008698   
Consumer Discretionary correlation_top_10             0.104542  0.037210   
                       lagged_correlation_top_10      0.097480  0.018148   
                       pcmi+                          0.087947  0.012483   
Consumer Staples       correlation_top_10             0.061084  0.010865   
                       lagged_correlation_top_10      0.042644  0.002778   
                       pcmi+                          0.082942  0.010761   
Energy                 correlation_top_10             0.129593  0.027410   
                       lagged_correlation_top_10      0.145582  0.034951   
                       pcmi+                          0.125020  0.027851   
Financials             correlation_top_10             0.073891  0.012077   
                       lagged_correlation_top_10      0.062088  0.007174   
                       pcmi+                          0.046346  0.003531   
Health Care            correlation_top_10             0.064100  0.007684   
                       lagged_correlation_top_10      0.083584  0.009555   
                       pcmi+                          0.085609  0.015107   
Industrials            correlation_top_10             0.168285  0.086368   
                       lagged_correlation_top_10      0.060144  0.007097   
                       pcmi+                          0.122656  0.029983   
Information Technology correlation_top_10             0.068057  0.009944   
                       lagged_correlation_top_10      0.077343  0.011221   
                       pcmi+                          0.099837  0.019049   
Materials              correlation_top_10             0.088784  0.015347   
                       lagged_correlation_top_10      0.097203  0.028432   
                       pcmi+                          0.073400  0.007969   
Real Estate            correlation_top_10             0.242333  0.131838   
                       lagged_correlation_top_10      0.117346  0.025564   
                       pcmi+                          0.129948  0.026550   
Utilities              correlation_top_10             0.043219  0.002631   
                       lagged_correlation_top_10      0.040160  0.002263   
                       pcmi+                          0.049494  0.003609   

predictive_model                                           linear_regression  \
                                                      rmse               mae   
sector                 model                                                   
Communication Services correlation_top_10         0.116398          0.068412   
                       lagged_correlation_top_10  0.148429          0.085504   
                       pcmi+                      0.093265          0.085198   
Consumer Discretionary correlation_top_10         0.192899          0.092065   
                       lagged_correlation_top_10  0.134715          0.094690   
                       pcmi+                      0.111726          0.084339   
Consumer Staples       correlation_top_10         0.104234          0.046353   
                       lagged_correlation_top_10  0.052706          0.053405   
                       pcmi+                      0.103737          0.050484   
Energy                 correlation_top_10         0.165560          0.360219   
                       lagged_correlation_top_10  0.186953          0.207109   
                       pcmi+                      0.166885          0.155922   
Financials             correlation_top_10         0.109893     

In [60]:
latex_output = pivot_df.round(3).to_latex(multirow=True,float_format="%.3f")
latex_output = latex_output.replace('_', '\_')